In [ ]:
# pip install 'sagemaker[local]' 'docker' --upgrade

In [1]:
import sagemaker
from sagemaker.local import LocalSession

# sagemaker_session = LocalSession()
# sagemaker_session.config = {'local': {'local_code': True}}

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:
from sagemaker.tensorflow.serving import Model

model_path = 's3://sagemaker-us-east-2-280662875630/sagemaker-tensorflow-scriptmode-2020-04-30-17-58-04-177/output/model.tar.gz'

model = Model(model_data=model_path, role=role)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


# Local prediction

In [3]:
# local_predictor = model.deploy(initial_instance_count=1, instance_type='local_gpu')

In [4]:
#tf_predictor = model.deploy(initial_instance_count=1,
#                                   instance_type='ml.p2.xlarge')      # $1.361/hour in eu-west-1

tf_predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.c5.large',        # $0.134/hour in eu-west-1
                         accelerator_type='ml.eia1.medium',  # + $0.140/hour in eu-west-1
                         endpoint_name='shiptrack-detection')
#                          serializer=npy_serializer, deserializer=numpy_deserializer)     # = 80% discount!

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

In [6]:
# tf_predictor.content_type

NameError: name 'tf_predictor' is not defined

In [ ]:
# # This doesn't seem to work but I don't know why...
# from sagemaker.predictor import npy_serializer, numpy_deserializer

# tf_predictor.serializer = npy_serializer
# tf_predictor.deserializer = numpy_deserializer
# tf_predictor.content_type = npy_serializer.content_type
# tf_predictor.accept = npy_serializer.content_type

# I think this is the preffered way to do it: https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#create-python-scripts-for-custom-input-and-output-formats

In [5]:
# from sagemaker.tensorflow import TensorFlowPredictor
from sagemaker.predictor import RealTimePredictor

# predictor = TensorFlowPredictor('shiptrack-detection')
tf_predictor = RealTimePredictor('shiptrack-detection')
# result = predictor.predict(['my request body'])

In [8]:
import numpy as np
# normalise = lambda data: [(d - mean_data)/std_data for d in data]
# normalise = lambda data: (data - 0.5)/0.25
normalise = lambda data: [(d - 0.45)/0.25 for d in data]

def split_array(arr, step_size=440):
    x_len, y_len = arr.shape[0:2]
    x_split_locs = range(step_size, x_len, step_size)
    y_split_locs = range(step_size, y_len, step_size)
    v_splits = np.array_split(arr, x_split_locs, axis=0)
    return sum((np.array_split(v_split, y_split_locs, axis=1) for v_split in v_splits), [])


def process_typed_file(f_key, image_size, path=None, resize=False, channel=None):
    import boto3
    from PIL import ImageOps, Image
    from io import BytesIO


    s3 = boto3.client('s3')
    file_byte_string = s3.get_object(Bucket='imiracli-data', Key=f_key)['Body'].read()

    im_data = Image.open(BytesIO(file_byte_string))
    if (im_data.size[1] not in [2030, 2040]) or (im_data.size[0] != 1354):
        print("Skipping wierd shape: {}".format(im_data.size))
        raise ValueError()
    
    original_shape = im_data.size
    
    if resize:
        im_data = im_data.resize(image_size, resample=Image.BILINEAR)
        padding = 0
    else:  # Pad
        # expand by 160, 80 on each side
        padding = 85 if im_data.size[1] == 2030 else 80
        im_data = ImageOps.expand(im_data, padding)

    # Drop the end of the y axis
    im_data = np.array(im_data)#[:, :1320, :]
    
    if channel is not None:
        im_data = im_data[..., channel]

    return im_data, original_shape


def combine_and_resize_masks(masks, original_size):
    print(masks.shape)
    # TODO: Make the nesting a parameter
    nested_masks = [np.split(m, 5) for m in np.split(masks[..., 0], 8, 0)]
#     print(len(nested_masks))
#     print(nested_masks[0][0].shape)
#     nested_masks = np.reshape(masks, (5, 8))
    
    mask = np.squeeze(np.block(nested_masks))
#     print(mask.shape)
    print(mask.any())
    mask_im = Image.fromarray(mask)

    mask_im = mask_im.resize(original_size, resample=Image.NEAREST)

    # Don't forget to pop off the superfluous color dimension
    new_mask = np.array(mask_im)#[:, :, 0]
    
    return new_mask


def get_ship_track_mask(f):
    data, original_shape = process_typed_file(f, (448*4, 448*3), resize=True)
    norm_data = normalise(data)
    print(original_shape)
    print(data.shape)

    split_data = normalise(split_array(np.concatenate([data[..., 0:1]/255.]*3, axis=-1), 448))
    # Normalise? Anythinf else?
#     print(split_data[0].shape)
    stacked_data = np.stack(split_data)
    print(stacked_data.shape)

    d = stacked_data[0:1,...] #.tolist()
    print(d)
    pred = tf_predictor.predict(data=d.tobytes())
    print(pred)
    prediction = np.array(pred['predictions'])
    
#     for p in prediction:
#         fig, axs = plt.subplots(figsize=(10, 20))
#         axs.imshow(data[..., 0])
#         axs.imshow(p[:,:, 0], alpha=0.5)
#         plt.show()
#     print(prediction.any())
#     print(prediction.shape)
    combined_prediction = combine_and_resize_masks(prediction, original_shape)
    
    combined_data = combine_and_resize_masks(np.stack(split_data), original_shape)
    
#     print(combined_data.shape)
#     print(combined_prediction.any())
#     print(combined_prediction)
    if combined_prediction.any():
        fig, axs = plt.subplots(figsize=(20, 40))
        axs.imshow(combined_data[...], vmin=-2, vmax=2)
        im=axs.imshow(combined_prediction[:,:], alpha=0.5, vmin=0, vmax=1)
#         plt.colorbar(im)
        plt.show()
    return combined_prediction

In [9]:
data_path = 's3://imiracli-data/MODIS_deep_cloud/test_niremi_inference_images/'
file_path = data_path+"images/MYD021KM.A2006166.1845.061.2018022234106.png"

arr = get_ship_track_mask("MODIS_deep_cloud/test_niremi_inference_images/images/MYD021KM.A2006166.1845.061.2018022234106.png")

(1354, 2030)
(1344, 1792, 2)
(12, 448, 448, 3)
[[[[ 0.03529412  0.03529412  0.03529412]
   [-0.35686275 -0.35686275 -0.35686275]
   [-0.63921569 -0.63921569 -0.63921569]
   ...
   [ 0.20784314  0.20784314  0.20784314]
   [ 0.23921569  0.23921569  0.23921569]
   [ 0.31764706  0.31764706  0.31764706]]

  [[ 0.30196078  0.30196078  0.30196078]
   [-0.23137255 -0.23137255 -0.23137255]
   [-0.65490196 -0.65490196 -0.65490196]
   ...
   [ 0.09803922  0.09803922  0.09803922]
   [ 0.34901961  0.34901961  0.34901961]
   [ 0.36470588  0.36470588  0.36470588]]

  [[ 1.29019608  1.29019608  1.29019608]
   [ 1.03921569  1.03921569  1.03921569]
   [ 0.94509804  0.94509804  0.94509804]
   ...
   [-0.02745098 -0.02745098 -0.02745098]
   [ 0.23921569  0.23921569  0.23921569]
   [ 0.20784314  0.20784314  0.20784314]]

  ...

  [[ 1.05490196  1.05490196  1.05490196]
   [ 1.16470588  1.16470588  1.16470588]
   [ 1.11764706  1.11764706  1.11764706]
   ...
   [-1.21960784 -1.21960784 -1.21960784]
   [-1.266

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (415) from model with message "{"error": "Unsupported Media Type: Unknown"}". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/shiptrack-detection in account 280662875630 for more information.

In [ ]:
help(tf_estimator.model_data)

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = tf_predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_label))

In [12]:
sess.delete_endpoint(endpoint_name='shiptrack-detection')